# Expected Completion

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist #function to create adjacency matrix

In [2]:
plays = pd.read_csv("data/pass_receiver.csv")
pff = pd.read_csv("data/pffScoutingData.csv")


read in weekly tracking files and concatenate into one tracking file, then delete weeklies

In [3]:
week1 = pd.read_csv("data/week1.csv")
week2 = pd.read_csv("data/week2.csv")
week3 = pd.read_csv("data/week3.csv")
week4 = pd.read_csv("data/week4.csv")
week5 = pd.read_csv("data/week5.csv")
week6 = pd.read_csv("data/week6.csv")
week7 = pd.read_csv("data/week7.csv")
week8 = pd.read_csv("data/week8.csv")
tracking = pd.concat([week1,week2,week3,week4,week5,week6,week7,week8])


In [4]:
#save memory
del week1,week2,week3,week4,week5,week6,week7,week8

In [5]:
tracking.columns

Index(['gameId', 'playId', 'nflId', 'frameId', 'time', 'jerseyNumber', 'team',
       'playDirection', 'x', 'y', 's', 'a', 'dis', 'o', 'dir', 'event'],
      dtype='object')

add unique identifier of each play and frame (some playIds repeat over games)

In [6]:
tracking["comb_id"] = tracking["gameId"].astype("str")+" "+tracking["playId"].astype("str")
pff["comb_id"] = pff["gameId"].astype("str")+" "+pff["playId"].astype("str")
plays["comb_id"] = plays["gameId"].astype("str")+" "+plays["playId"].astype("str")
tracking["comb_and_frame"] = tracking["gameId"].astype("str")+" "+tracking["playId"].astype("str")+" "+tracking["frameId"].astype("str")

isolate frames at time of pass

In [7]:
#isolate just forward passes and leave out a certain frame (duplicate forward pass for a play)
just_forward_passes = tracking[(tracking["event"]=="pass_forward") & (tracking["comb_and_frame"]!="2021091203 3740 32")& (tracking["team"]!="football")]
#bring in receiver info from plays
tracking_plays =just_forward_passes.merge(plays[["comb_id","possessionTeam","is_pass_failed","PassReceiver_nflId"]], how="left",on="comb_id" )
#and qb info from pff
tracking_merged_all = tracking_plays.merge(pff[["comb_id","nflId","pff_role"]], how = "left",on=["comb_id","nflId"])

In [8]:
len(just_forward_passes)

166034

In [9]:
len(tracking_plays)

166276

isolate tracking to just QB, OL, receiver and defense

In [10]:
tracking_players_filtered = tracking_merged_all[(tracking_merged_all["nflId"]==tracking_merged_all["PassReceiver_nflId"])|(tracking_merged_all["pff_role"]=="Pass")|(tracking_merged_all["pff_role"]=="Pass Block")|(tracking_merged_all["team"]!=tracking_merged_all["possessionTeam"])]

# Function to take in 1 pass play and put it in graph format( list containing adjacency matrix, features, y) this will then be parallelized for all passes in the data

In [78]:
sample = tracking_players_filtered[tracking_players_filtered.comb_id == "2021090900 97"]
sample_dist = sample[["x","y"]]

In [12]:
sample[sample["pff_role"]=="Pass"]

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,...,dis,o,dir,event,comb_id,comb_and_frame,possessionTeam,is_pass_failed,PassReceiver_nflId,pff_role
0,2021090900,97,25511.0,40,2021-09-10T00:26:35.000,12.0,TB,right,32.69,20.5,...,0.38,92.79,202.55,pass_forward,2021090900 97,2021090900 97 40,TB,1,44896.0,Pass


In [82]:
def angle_relative_diff(a,b):
    diff_list = []
    for i in range(len(a)):
        difference = a[i] - b[0]
        if difference < -180:
            difference +=360
        elif difference > 180:
            difference -=360
        diff_list.append(difference)
    return diff_list

In [81]:
len(sample["o"])

18

In [83]:
angle_relative_diff(sample["o"],QB["o"])

KeyError: 1

In [74]:
sample["onPossessionTeam"] = np.where(sample["possessionTeam"]==sample["team"],1,0)
QB= sample[sample["pff_role"]=="Pass"]
sample["rel_o"] = sample["o"]-QB["o"][0]
sample["rel_dir"] = sample["dir"]-QB["dir"][0]
sample = sample[["onPossessionTeam","rel_dir","rel_o","pff_role"]]
one_hot = pd.get_dummies(sample['pff_role'])
sample= sample.drop('pff_role',axis = 1)
# Join the encoded df
sample = sample.join(one_hot)



C:\Users\conny\AppData\Local\Temp\ipykernel_20712\2541114226.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample["onPossessionTeam"] = np.where(sample["possessionTeam"]==sample["team"],1,0)
C:\Users\conny\AppData\Local\Temp\ipykernel_20712\2541114226.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample["rel_o"] = sample["o"]-QB["o"][0]
C:\Users\conny\AppData\Local\Temp\ipykernel_20712\2541114226.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [ ]:
def one_play_graph_function(data):
    one_play_formatted_list = []
    sample = sample.reset_index()
    #first put play in adjacency matrix format, the distance of all players to each other (this will then be filtered to just link relevant players to QB or WR)
    adj = cdist(sample_dist.values, sample_dist.values)
    for index,row in sample.iterrows():
        #connect QB to pass rush/block and wr, WR to coverage and QB, remove other connections by setting to zero
        if (row["pff_role"]=="Coverage" )| (row["pff_role"]=="Pass Route"):
            condition = ((sample["pff_role"]=="Pass Block")|(sample["pff_role"]=="Pass Rush"))
        else:
            condition = ((sample["pff_role"]=="Coverage"))
        adj[index][condition] = 0
    #then put play in features format, getting a table containing position, speed, orientation etc
    sample["onPossessionTeam"] = np.where(sample["possessionTeam"]==sample["team"],1,0)
    QB= sample[sample["pff_role"]=="Pass"]
    sample["rel_o"] = sample["o"]-QB["o"][0]
    sample["rel_dir"] = sample["dir"]-QB["dir"][0]
    sample = sample[["onPossessionTeam","rel_dir","rel_o","pff_role"]]
    one_hot = pd.get_dummies(sample['pff_role'])
    sample= sample.drop('pff_role',axis = 1)
    # Join the encoded df
    sample = sample.join(one_hot)
    #then get y, the response variable for this play (binary pass completed or not)
    return one_play_formatted_list